In [ ]:
#https://stackoverflow.com/a/52954738
# path may (will) vary from user to user
import os
#pascal:
os.environ['PROJ_LIB'] = '/Users/PascalUZH/opt/miniconda3/envs/myNewCondaEnv/share/proj'


In [ ]:
import math
import copy
import numpy as np
import scipy as sp
import networkx as nx
import networkx.algorithms.cluster as nx_clust
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.rcParams['text.usetex'] = True
from matplotlib import cm
import pandas as pd
from mpl_toolkits.basemap import Basemap
import seaborn as sns

In [ ]:
# read stored data from data_preparation notebook
%store -r df_airports
%store -r airports_dict
%store -r df_merged

G = nx.read_gml('Graphs/airlines.gml')

el PLAN (Small World):
- derive metrics which are required for building a null model
- build Small World compatible null model
    (small world property: <d> ~ log N / log<k>)
    - Watts-Strogatz model (Ex 4.3 -> nx.watts_strogatz_graph)
        - does NOT consider/simulate network GROWTH, but rather for analysis
        of underlying structure
        - explicitly defined as used for building SW graphs
        - N := nr. of nodes
        - k := nr. of nearest neighbours a node is initially connected to
        - p := rewiring probability
    - we may want to consider the connected_watts_strogatz_graph(), as the "regular"
    one is not guaranteed to be connected
        - takes additional param 'tries', which specifies how many times it is
        allowed to run the "regular" algo win search of a connected graph

el PLAN (scale free -> do in different notebook)
- will be with albert barabasi as null model

In [ ]:
## metrics to be used in construction of null model
# k := nr. of connections per node in initial topology
#   -> set to avg. degree of G
#   -> use math.ceil to round up because documentation states:
#       k will be k-1 if k is odd
#           thus if k starts odd (11.49) and we round down, the difference will be larger (1.49, min. 1.01) than if we simply
#           round up (.51, max .99).
#           in case it starts even (10.01) and we round up, the resulting k will be 10 anyway.
#   -> since WS is undirected and G is directed, we have to double k
# p := rewiring probability, element of [0, 1]
#   http://www.scholarpedia.org/article/Small-world_network
#       -> "Because the WS model is define probabilistically for p ≠ 0, studying
#           entails examining an ensemble of graphs for each p (for a fixed value of N).
#           When one states a property of the WS model for a fixed set of parameter
#           values (and with p ≠ 0), it is to be understood as a mean over an ensemble
#           of graphs (aside from properties that are fixed a priori for each groph
#           from the definition of the ensemble"
#   -> have to input a range of ps.
#       - arbitrary choice: some large enough nr. of graphs, evenly spaced probabilities
N = nx.number_of_nodes(G)
avg_deg = nx.number_of_edges(G) / float(G.number_of_nodes())
k = math.ceil(2 * avg_deg)
p_range = np.linspace(0, 1, 100)

    for completeness, let us first analyze (as in exercise) whether the WS model we
    generate exhibits the SW properties we would expect given the below analytical
    solution

In [ ]:
p_0 = 0
clustering_0_analytical = (3/2) * ((k-1)/(2*k-1)) * np.power((1-p_0),3)
clustering_0_observed = nx_clust.average_clustering(nx.watts_strogatz_graph(N, k, p_0))

def avg_clustering_coefficient_analytical(p):
    return (3/2) * ((k-1)/(2*k-1)) * np.power((1-p),3) / clustering_0_analytical

clustering_per_p = []
clustering_analytical_per_p = []
avg_shortest_path_per_p = []


for p in p_range:
    # for each p in the defined list, we have to examine a
    # specific watt-strogatz graph
    G_WS = nx.watts_strogatz_graph(N, k, p)

    avg_clust = nx_clust.average_clustering(G_WS) / clustering_0_observed
    avg_clust_analytical = avg_clustering_coefficient_analytical(p)

    clustering_per_p.append(avg_clust)
    clustering_analytical_per_p.append(avg_clust_analytical)

    # also calculate the average shortest path
    avg_sp = nx.average_shortest_path_length(G_WS)
    avg_shortest_path_per_p.append(avg_sp)

In [ ]:
# plot for comparison
# TODO: proper latex labels -> see exercise

fig1 = plt.figure(dpi=500)
plt.rcParams["figure.figsize"] = (10, 4)
plt.plot(p_range, clustering_per_p, label="observed in WS model")
plt.plot(p_range, clustering_analytical_per_p, label="analytical result", color='red', alpha=0.9)
plt.xlabel('rewiring probability p')
plt.ylabel('average clustering C(p)/C(0)')
plt.title('Average clustering in the Watts-Strogatz model')
# plt.xscale('log')
# TODO: maybe logscale?
plt.legend()
plt.show()

In [ ]:
# calculate average error (abs)
err_clust_WS = list(np.abs(np.array(clustering_per_p)-np.array(clustering_analytical_per_p)))
err_clust_WS_avg = sum(err_clust_WS)/len(err_clust_WS)
print(f"average error (abs) of WS model to analytical result: {err_clust_WS_avg}")

In [ ]:
# calculate small world property for air traffic graph
# <d> ~ log N / log <k>

def expected_small_world_path_length(N, avg_deg):
    return np.log(N) / np.log(avg_deg)

# create a list of disconnected nodes (breaks avg. shortest path length)
# disconnected_nodes = list(nx.isolates(G))
# avg_shortest_path = nx.average_shortest_path_length(G)

# manually calculate avg. shortest path, because the nx.average_shortest_path_length does not
# work for directed graphs
shortest_path_per_node = list(nx.shortest_path_length(G))
avg_shortest_path_per_node = []
for node in shortest_path_per_node:
    path_dict = node[1]
    path_length_sum = 0
    for _, v in path_dict.items():
        path_length_sum += v
    # if len = 1 the path_dict only contains the node itself with 0 length path entry.
    # exclude those entries
    if len(path_dict) != 1:
        # subtract one, because the path_dict contains the node itself as well
        avg_spl = path_length_sum / (len(path_dict)-1)
        avg_shortest_path_per_node.append(avg_spl)
avg_shortest_path = sum(avg_shortest_path_per_node) / len(avg_shortest_path_per_node)

# calculate expected small world path length
avg_shortest_path_expected = expected_small_world_path_length(N, avg_deg)

In [ ]:
# also calculate the avg. avg. shortest path of the set of WS graphs for comparison
# (yes, 'average average shortest path' is correct)
avg_shortest_path_WS_model = sum(avg_shortest_path_per_p) / len(avg_shortest_path_per_p)
